In [1]:
from aocd import get_data, submit
import re
import numpy as np
from dataclasses import dataclass

In [2]:
example_data = "Monkey 0:\n  Starting items: 79, 98\n  Operation: new = old * 19\n  Test: divisible by 23\n    If true: throw to monkey 2\n    If false: throw to monkey 3\n\nMonkey 1:\n  Starting items: 54, 65, 75, 74\n  Operation: new = old + 6\n  Test: divisible by 19\n    If true: throw to monkey 2\n    If false: throw to monkey 0\n\nMonkey 2:\n  Starting items: 79, 60, 97\n  Operation: new = old * old\n  Test: divisible by 13\n    If true: throw to monkey 1\n    If false: throw to monkey 3\n\nMonkey 3:\n  Starting items: 74\n  Operation: new = old + 3\n  Test: divisible by 17\n    If true: throw to monkey 0\n    If false: throw to monkey 1"
real_data = get_data(day=11, year=2022)

In [3]:
class Monkey:
    def __init__(self, starting_items, operation_raw, relieve, test_divisible_by, throw_to_monkey_true, throw_to_monkey_false, monkeys):
        self.items: [] = starting_items
        self.operation_raw = operation_raw
        self.relieve = relieve
        self.test_divisible_by = test_divisible_by
        self.throw_to_monkey_true = throw_to_monkey_true
        self.throw_to_monkey_false = throw_to_monkey_false
        self.monkeys = monkeys
        self.inspected_items = 0

    def receive_item(self, worry_level):
        self.items.append(worry_level)

    def operation(self, old):
        return eval(self.operation_raw)

    def has_items(self):
        return len(self.items) > 0

    def get_items(self):
        return self.items

    def inspect_and_throw(self):
        self.inspected_items = self.inspected_items + 1
        worry_level = self.items.pop(0)
        worry_level = self.operation(worry_level)
        worry_level = self.relieve(worry_level)

        throw_to_monkey = self.throw_to_monkey_true if worry_level % self.test_divisible_by == 0 else self.throw_to_monkey_false
        self.monkeys[throw_to_monkey].receive_item(worry_level)


In [4]:
def simulate(part, rounds):
    monkeys = []
    divisible_list = []
    for monkey_raw in [x.splitlines() for x in real_data.split("\n\n")]:
        divisible_list.append(int(monkey_raw[3].strip()[19:]))

    prod = np.prod(divisible_list)

    for monkey_raw in [x.splitlines() for x in real_data.split("\n\n")]:
        starting_items = list(map(np.int64, monkey_raw[1].strip()[16:].split(",")))
        operation_raw = monkey_raw[2].strip()[17:]
        test_divisible_by = int(monkey_raw[3].strip()[19:])
        throw_to_monkey_true = int(monkey_raw[4].strip()[25:])
        throw_to_monkey_false = int(monkey_raw[5].strip()[25:])
        relieve = (lambda each: each % prod) if part==2 else  (lambda each: each // 3)
        monkeys.append(Monkey(starting_items, operation_raw, relieve, test_divisible_by, throw_to_monkey_true, throw_to_monkey_false, monkeys))

    for r in range(rounds):
        for monkey in monkeys:
            while monkey.has_items():
                monkey.inspect_and_throw()

    print(f"Monkey business for part {part}: {np.prod(sorted([x.inspected_items for x in monkeys])[-2:], dtype=np.int64)}")

In [5]:
simulate(1, 20)

Monkey business for part 1: 67830


In [6]:
simulate(2, 10000)

Monkey business for part 2: 15305381442
